### Implementing FunkSVD - Solution

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('../data/movies_clean.csv')
reviews = pd.read_csv('../data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the funtion on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of matrix factorization.

In [2]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting **U** and **V** matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [3]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 48.464341
2 		 19.152872
3 		 5.072273
4 		 2.903386
5 		 2.702262
6 		 2.658502
7 		 2.630656
8 		 2.603075
9 		 2.572771
10 		 2.538804


In [4]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.28729394   9.18159883  10.3011113   10.10854516]
 [  8.40901963   7.25897547   9.17647614   8.96236641]
 [  7.96434463   6.96572226   8.29078269   7.62868227]
 [  9.49745495   8.16283023  10.0978691    9.6359178 ]
 [  8.44627578   7.27872196   9.0380299    8.64455622]
 [  6.93472885   6.11365411   7.10425854   6.83177471]
 [  9.13015257   7.93629198   9.75497583   9.22649788]
 [  8.24952963   7.03525062   8.87468261   8.03327882]
 [  8.30034121   7.13815678   9.19308563   8.50367206]
 [  7.75389741   6.62928807   8.24367755   7.5336925 ]
 [  8.90593631   7.57443516   9.54381082   8.76562802]
 [  9.68793559   8.53054925   9.9485221    9.55102636]
 [  9.29353082   8.45644298   9.40382861   9.31821645]
 [  6.64271593   5.7035421    7.01594244   6.62622346]
 [  9.60466595   8.29849156  10.21659348   9.96836171]
 [  9.88644628   8.80495757  10.09719035   9.61751354]
 [  8.35964657   7.52516593   8.2682908    8.46863883]
 [  7.33621077   6.63868097   7.08323205   7.0246992 ]
 [  8.2525

**The predicted ratings from the dot product are already starting to look a lot like the original data values even after only 10 iterations.  You can see some extreme low values that are not captured well yet.  The 5 in the second to last row in the first column is predicted as an 8, and the 4 in the second row and second column is predicted to be a 7.  Clearly the model is not done learning, but things are looking good.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting **U** and **V** matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [5]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 41.541069
2 		 13.482928
3 		 3.816577
4 		 2.809316
5 		 2.701528
6 		 2.657173
7 		 2.621357
8 		 2.583958
9 		 2.541877
10 		 2.493690
11 		 2.438410
12 		 2.375254
13 		 2.303660
14 		 2.223379
15 		 2.134590
16 		 2.038004
17 		 1.934932
18 		 1.827282
19 		 1.717461
20 		 1.608176
21 		 1.502150
22 		 1.401817
23 		 1.309049
24 		 1.224998
25 		 1.150071
26 		 1.084029
27 		 1.026167
28 		 0.975528
29 		 0.931074
30 		 0.891823
31 		 0.856916
32 		 0.825642
33 		 0.797428
34 		 0.771816
35 		 0.748433
36 		 0.726966
37 		 0.707140
38 		 0.688703
39 		 0.671425
40 		 0.655085
41 		 0.639475
42 		 0.624401
43 		 0.609680
44 		 0.595143
45 		 0.580637
46 		 0.566024
47 		 0.551182
48 		 0.536002
49 		 0.520396
50 		 0.504288
51 		 0.487622
52 		 0.470359
53 		 0.452479
54 		 0.433982
55 		 0.414890
56 		 0.395245
57 		 0.375112
58 		 0.354576
59 		 0.333746
60 		 0.312745
61 		 0.291713
62 		 0.270803
63 		 0.250170
64 		

In [6]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.00001034  10.00000426  10.00000266   9.99999   ]
 [ 10.000003     4.00000123   9.00000072   9.99999696]
 [  7.9999992    8.99999966   9.99999972   5.00000059]
 [  9.00002554   8.0000101   10.00000621   9.99997548]
 [ 10.00000495   5.00000157   9.00000077   8.99999507]
 [  5.99998704   3.99999452   9.99999642   6.00001218]
 [  9.00000884   8.00000318  10.0000018    8.99999138]
 [  9.99999043   4.99999585   8.99999723   8.00000891]
 [  6.99999846   7.99999908   9.99999928   8.00000132]
 [  8.99999668   4.99999857   8.99999903   7.00000298]
 [  9.00001087   8.00000417  10.00000248   7.99998945]
 [  9.00000574  10.00000183  10.00000093   8.99999434]
 [  9.99998068   8.99999188   9.99999474   8.00001823]
 [  4.9999863    7.99999421   4.99999628   8.00001303]
 [  9.99999858   7.99999952   9.99999968  10.00000119]
 [  8.99999792   8.99999925   9.99999952  10.00000186]
 [  8.99999694   7.99999892   7.99999934   8.00000279]
 [  9.99999728   7.99999889   0.99999929  10.00000257]
 [  4.9999

**In this case, we were able to completely reconstruct the item-movie matrix to obtain an essentially 0 mean squared error. I obtained 0 MSE on iteration 165.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [7]:
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [8]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 44.864034
2 		 17.146488
3 		 4.949918
4 		 3.077611
5 		 2.845885
6 		 2.777227
7 		 2.734186
8 		 2.696149
9 		 2.657806
10 		 2.616997
11 		 2.572363
12 		 2.522794
13 		 2.467306
14 		 2.405035
15 		 2.335286
16 		 2.257608
17 		 2.171887
18 		 2.078450
19 		 1.978149
20 		 1.872406
21 		 1.763188
22 		 1.652899
23 		 1.544178
24 		 1.439634
25 		 1.341556
26 		 1.251678
27 		 1.171029
28 		 1.099913
29 		 1.038000
30 		 0.984485
31 		 0.938283
32 		 0.898195
33 		 0.863038
34 		 0.831725
35 		 0.803300
36 		 0.776947
37 		 0.751982
38 		 0.727829
39 		 0.704012
40 		 0.680129
41 		 0.655852
42 		 0.630912
43 		 0.605102
44 		 0.578273
45 		 0.550343
46 		 0.521291
47 		 0.491170
48 		 0.460100
49 		 0.428273
50 		 0.395944
51 		 0.363423
52 		 0.331062
53 		 0.299238
54 		 0.268327
55 		 0.238693
56 		 0.210656
57 		 0.184484
58 		 0.160375
59 		 0.138452
60 		 0.118764
61 		 0.101292
62 		 0.085957
63 		 0.072632
64 		

In [9]:
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 11.252904946215294:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [10]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 23.043054
2 		 10.624768
3 		 7.314082
4 		 5.657515
5 		 4.612804
6 		 3.880947
7 		 3.335650
8 		 2.912864
9 		 2.575807
10 		 2.301409
11 		 2.074291
12 		 1.883753
13 		 1.722120
14 		 1.583750
15 		 1.464408
16 		 1.360851
17 		 1.270545
18 		 1.191467
19 		 1.121970
20 		 1.060690


`6.` Now that you have a set of predictions for each user-movie pair.  Let's answer a few questions about your results. Provide the correct values to each of the variables below, and check your solutions using the tests below.

In [11]:
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [12]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
